### **Modelos predictivos**

#### **Librerias:**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from joblib import dump, load

- Importamos el dataset

In [2]:
original = pd.read_csv('coches-de-segunda-mano-sample.csv')
df = original.copy()

In [3]:
df.sample(1)

,url,company,make,model,version,price,price_financed,fuel,year,kms,...,doors,shift,color,photos,is_professional,dealer,province,country,publish_date,insert_date
24329,3640eae75e7a0f9cf48cf050f2b49403,9881bcdd5a0ad4733037b3fb25e69c3a,NISSAN,JUKE,NISSAN JUKE DIGT E6 140kW190CV XTRONIC 4x4i TE...,16990,15790.0,Gasolina,2017.0,5776,...,5,Automático,Gris / Plata,37,True,Flexicar Valencia,Valencia,Spain,2020-12-28 12:55:07,2021-01-15 00:00:00


**EDA**

- Borramos las dos primeras columnas y las dos finales
- Miramos duplicados limpiando la data "dropeando" cuatro columnas.
- Las columnas eliminadas son columnas irrelevantes para el analisis ya que da erroneo el momento de mirar duplicados.

In [4]:
url_comp = ["url","company","publish_date","insert_date"]

In [5]:
df2 = df.drop(columns = url_comp, inplace = False)
df2.sample(5)

,make,model,version,price,price_financed,fuel,year,kms,power,doors,shift,color,photos,is_professional,dealer,province,country
26999,SEAT,Tarraco,SEAT Tarraco 2.0 TDI 110kW 4Drive DSG SS FR GO...,39900,36900.0,Diésel,2020.0,14438,150.0,5,Automático,Blanco,28,True,Huelva Motor,Huelva,Spain
22042,VOLKSWAGEN,Passat,VOLKSWAGEN Passat BlueMotion 1.6 TDI 120CV 4p.,13900,13400.0,Diésel,2016.0,145000,120.0,4,Manual,Gris / Plata,11,True,Autos Huertas,Córdoba,Spain
20668,RENAULT,Laguna,RENAULT Laguna Limited dCi 110 eco2 5p.,7500,7500.0,Diésel,2015.0,210000,110.0,5,Manual,Blanco,11,True,Jose Antonio Munuera Soto,Murcia,Spain
48022,RENAULT,Laguna,RENAULT Laguna 5p.,1800,NaN,Gasolina,2001.0,175000,NaN,5,Automático,Verde,7,False,3b948865c68fbebbdfce382e5eb95169,A Coruña,Spain
36743,VOLKSWAGEN,Golf,VOLKSWAGEN Golf GTD 2.0 TDI 184cv BMT 5p.,18990,NaN,Diésel,2015.0,99980,184.0,5,Manual,Gris / Plata (carbon steel ),40,True,R&R Sports,La Rioja,Spain


In [6]:
# Primero almacenamos las columnas que no son importantes para el analisis
columns_to_drop = ['url', 'company', 'publish_date', 'insert_date']
# Almacenamos en a el df sin esas columnas

# Utilizamos una funcion para eliminar los duplicados en base al df a
def val_duplicated(df):
    import pandas as pd
    if df.duplicated().sum() != 0:
        print(f"Hay un total de {df.duplicated().sum()} valores duplicados")
        return df.drop_duplicates(inplace=True)
    else:
        print("No hay valores duplicados")
        return df
#Llamamos a la funcion para saber cuantos duplicados tenemos
val_duplicated(df2)

Hay un total de 101 valores duplicados


In [7]:
df2.shape

(49899, 17)

- Los valores nulos de precio financiado los cambiamos a 0

In [8]:
df2['price_financed'].fillna(0, inplace = True)

In [9]:
df2.isnull().sum()

make                  2
model                 5
version               0
price                 0
price_financed        0
fuel                 46
year                  2
kms                   0
power              8513
doors                 0
shift               111
color               594
photos                0
is_professional       0
dealer                0
province              6
country               0
dtype: int64

- Eliminamos todas las columnas con valores nulos menos la de "power"

In [10]:
df2.drop(['make', 'model', 'fuel','year','shift','color','province','country','dealer'], axis=1, inplace = True)

In [11]:
df2.isnull().sum()

version               0
price                 0
price_financed        0
kms                   0
power              8513
doors                 0
photos                0
is_professional       0
dtype: int64

**Outliers**

In [12]:
# Tu código aquí debajo
numericas=['price','price_financed','kms','power','doors']

# FUNCIÓN QUE DEVUELVE LOS NÚMEROS DE FILA EN LOS QUE LA VARIABLE EN CUESTIÓN ES OULIER
def outliers(var):
    q1=var.quantile(0.25)
    q3=var.quantile(0.75)
    riq=q3-q1
    sup=q3+1.5*(riq)
    inf=q1-1.5*(riq)
    outl=(var>sup) | (var<inf)
    return outl

for i in df2.loc[:,numericas]:
    print("Cantida de oultliers de la variable",i,":",outliers(df2[i]).sum())
# Eliminar outliers
# Tu código
for i in df2.loc[:,numericas]:
    df2=df2[~outliers(df[i])]
df2.sample(5)

Cantida de oultliers de la variable price : 2203
Cantida de oultliers de la variable price_financed : 1489
Cantida de oultliers de la variable kms : 479
Cantida de oultliers de la variable power : 3273
Cantida de oultliers de la variable doors : 2480


C:\Users\GuillemEdoBertran\AppData\Local\Temp\ipykernel_2304\1543499220.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2=df2[~outliers(df[i])]
C:\Users\GuillemEdoBertran\AppData\Local\Temp\ipykernel_2304\1543499220.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2=df2[~outliers(df[i])]
C:\Users\GuillemEdoBertran\AppData\Local\Temp\ipykernel_2304\1543499220.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2=df2[~outliers(df[i])]
C:\Users\GuillemEdoBertran\AppData\Local\Temp\ipykernel_2304\1543499220.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2=df2[~outliers(df[i])]
C:\Users\GuillemEdoBertran\AppData\Local\Temp\ipykernel_2304\1543499220.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2=df2[~outliers(df[i])]


,version,price,price_financed,kms,power,doors,photos,is_professional
33330,KIA,2200,0.0,284000,NaN,5,7,False
567,OPEL Zafira Enjoy 1.9 CDTi 8v 120 CV 5p.,4000,0.0,145000,120.0,5,10,True
20603,NISSAN NOTE 5p. 1.6 ACENTA 5p.,2595,0.0,179000,110.0,5,20,True
41506,BMW Serie%205 530D AUTO 4p.,1500,0.0,180000,184.0,4,4,False
35591,SEAT Exeo ST 2.0 TDI CR 143 CV DPF Style 5p.,5850,0.0,125540,143.0,5,8,False


In [13]:
df2.shape

(43673, 8)

## Creación modelo nº1

In [ ]:
# Eliminamos valores nulos de "power"

df2.dropna(df["power"])

**1.1 Variable a predecir = "power"**

One Hot Encoding para la columna "version"

In [14]:
onehot=OneHotEncoder()

# Aplico el OneHot a la columna "version" y guardo el resultao en "a"
a=onehot.fit_transform(df2[['version']])

# Convierto el a en data frame y lo llamo encoded_df
encoded_df2 = pd.DataFrame(a.toarray(), columns=onehot.get_feature_names_out(['version']))

# Reseteo el índice de los dos data frames antes de concatenarlos
df2.reset_index(drop = True, inplace = True)
encoded_df2.reset_index(drop = True, inplace = True)

# concateno los dos data frames y los guardo machacando el df
df2 = pd.concat([df2,encoded_df2], axis = 1)

# Elimino la columna "version" original antigua, la categórica
df2.drop(columns = 'version',inplace=True)

# Muestro tamaño y sample del data frame df2 transformado
print(df2.shape)
df2.sample(5)

(43673, 15057)


,price,price_financed,kms,power,doors,photos,is_professional,version_ D2S 27kW 3p.,version_ABARTH 500 1.4 16v 121kW Monster Energy Yamaha E6D 3p.,version_ABARTH 500 1.4 16v TJet 135cv 3p.,...,version_VOLVO XC90 2.5 T Momentum 5p.,version_VOLVO XC90 3.2 Summum 5p.,version_VOLVO XC90 D5 5p.,version_VOLVO XC90 D5 Executive 5p.,version_VOLVO XC90 D5 Kinetic 5p.,version_VOLVO XC90 D5 Momentum 5p.,version_VOLVO XC90 D5 RDesign Auto 7 Asientos 5p.,version_VOLVO XC90 D5 Summum 5p.,version_VOLVO XC90 D5 Summum Auto 7 Asientos 5p.,version_VOLVO XC90 D5 Volvo Ocean Race 5p.
14935,16995,14995.0,20300,130.0,5,13,True,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23012,23800,0.0,132000,190.0,5,27,True,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28167,6900,0.0,152000,85.0,5,9,True,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43624,6900,0.0,210000,218.0,5,8,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20548,8490,0.0,148648,90.0,5,14,True,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Antes de dropear la columna, hay que borrar todos los nulos de "power" para que lo pueda predecir.

X=df2.drop(columns=['power'],inplace=False)
y=df2['power']

In [16]:
my_kfold = KFold(n_splits = 30)

In [17]:
RF = RandomForestClassifier(n_estimators = 100)
estandarizador = StandardScaler()
accuracies=[]
precisions=[]
recalls=[]
f1s=[]

for train_index, test_index in my_kfold.split(df2):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    estandarizador.fit(X_train)
    X_train_std=estandarizador.transform(X_train)
    X_test_std=estandarizador.transform(X_test)
    RF.fit(X_train,y_train)
    
    predictions = RF.predict(X_test)
    accuracies.append(accuracy_score(y_test, predictions))
    precisions.append(precision_score(y_test, predictions))
    recalls.append(recall_score(y_test, predictions))
    f1s.append(f1_score(y_test, predictions))

ValueError: Input y contains NaN.

In [ ]:
metricas_RF=[]
metricas_RF.append(sum(accuracies)/len(accuracies))
metricas_RF.append(sum(precisions)/len(precisions))
metricas_RF.append(sum(recalls)/len(recalls))
metricas_RF.append(sum(f1s)/len(f1s))
metricas_RF

In [ ]:
dict={"RandomForest":metricas_RF}
Resultados=pd.DataFrame(dict,index=["Acuracy","Precision","Recall","F1"])
Resultados